# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
from rasterstats import zonal_stats
import fiona
from rasterio.plot import show
from rasterio.mask import mask
import rasterstats
import shapely.geometry as geom
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


In [2]:
gdf = gpd.read_file("../outputs/all_env_factors.gpkg")
gdf.head()

,fire_id,fire_year,fire_month,fire_day,fire_rep_date,fire_size_ha,fire_calc_ha,fire_cause,fire_map_method,fire_poly_date,...,climate_P,climate_S,climate_SG,Year,Month,climate_long,climate_lat,climate_RH,land_cover_class,geometry
0,2004-C10176,2004,6,23,2004-06-23,520.7,520.796287,L,digitized,2007-05-17,...,17.1,0.0,0.0,2025,5,-124.136,52.114,43.20,Unclassified,POINT (1103487.457 895235.59)
1,2004-C50114,2004,6,20,2004-06-20,268.2,268.290572,L,digitized,2007-05-17,...,17.1,0.0,0.0,2025,5,-124.136,52.114,43.20,Developed,POINT (1059804.998 790281.638)
2,2004-C50125,2004,6,21,2004-06-21,20506.4,20506.415129,L,Modified from Protection,2007-05-17,...,80.5,0.0,0.0,2020,6,-126.587,52.389,51.68,Unclassified,POINT (1027509.177 798970.739)
3,2004-C50149,2004,6,22,2004-06-22,2408.5,2408.587142,L,digitized,2007-05-17,...,80.5,0.0,0.0,2020,6,-126.587,52.389,51.68,Unclassified,POINT (980057.152 883141.445)
4,2004-C50508,2004,8,16,2004-08-16,110.7,110.797330,L,digitized,2007-05-17,...,80.5,0.0,0.0,2020,6,-126.587,52.389,51.68,Rock/Rubble,POINT (981546.966 867012.646)


In [3]:
gdf = gdf.rename(columns={
    "Year": "climate_year",
    "Month": "climate_month"
    })

In [4]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 29955 entries, 0 to 29954
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fire_id           29955 non-null  object        
 1   fire_year         29955 non-null  int64         
 2   fire_month        29955 non-null  int64         
 3   fire_day          29955 non-null  int64         
 4   fire_rep_date     29955 non-null  datetime64[ms]
 5   fire_size_ha      29955 non-null  float64       
 6   fire_calc_ha      29955 non-null  float64       
 7   fire_cause        29955 non-null  object        
 8   fire_map_method   29935 non-null  object        
 9   fire_poly_date    28660 non-null  datetime64[ms]
 10  climate_index     29955 non-null  int64         
 11  dist_to_station   29955 non-null  float64       
 12  dist_roads        29955 non-null  float64       
 13  overlap_pct       29955 non-null  float64       
 14  dist_community

---

### Choosing the criteria for feature creation (Classification)

1. Log-transform and scale skewed continuous variables
    * Step 1 – Fire Size Scaling:

<code>The idea here is to log-transformed fire_size_sqm to reduce skewness and then standardized it to bring values on a comparable scale with other features.</code>

In [59]:
# Import needed libraries
from sklearn.preprocessing import MinMaxScaler

# Make a copy of the GeoDataFrame
restoration_dataset = gdf.copy()

# Log-transform fire size (skewed distribution)
restoration_dataset["fire_size_sqm_log"] = np.log1p(restoration_dataset["fire_size_sqm"])

restoration_dataset["climate_P_log"] = np.log1p(restoration_dataset["climate_P"])

# Initialize the scaler

scaler = MinMaxScaler()

# Continuous features
cont_features = [
    "climate_Tm", "climate_Tx", "climate_Tn",
    "climate_P", "climate_S", "climate_SG", "climate_RH",
    "elevation_mean", "elevation_std",
    "burnsev_mean", "burnsev_max", "burnsev_min",
    "fire_size_sqm", "dist_community", "dist_roads"
]

restoration_dataset[cont_features] = scaler.fit_transform(restoration_dataset[cont_features])
restoration_dataset["dist_community"] = 1 - restoration_dataset["dist_community"]
restoration_dataset["dist_roads"] = 1 - restoration_dataset["dist_roads"]



c:\Users\Dell\anaconda3\envs\fire_env\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


2. Scale burn severity and clip extreme values
    * Step 2 – Burn Severity:

<code>Extreme burn severity values were clipped at the 99th percentile, then standardized. This avoids outliers dominating the restoration score.</code>

In [60]:
# Burn severity columns
burnsev_features = ["burnsev_mean", "burnsev_max", "burnsev_min"]

# Clip extreme values at 99th percentile to avoid outliers
for col in burnsev_features:
    upper = restoration_dataset[col].quantile(0.99)
    restoration_dataset[col] = restoration_dataset[col].clip(upper=upper)

# Scale burn severity
restoration_dataset[burnsev_features] = scaler.fit_transform(restoration_dataset[burnsev_features])

3. Scale elevation and climate features
    * Step 3 – Elevation and Climate

<code>Standardization ensures temperature, precipitation, solar radiation, relative humidity, and elevation features are comparable. Precipitation is log-transformed due to skewness.</code>

In [61]:
# Elevation features
elev_features = ["elevation_mean", "elevation_std"]
restoration_dataset[elev_features] = scaler.fit_transform(restoration_dataset[elev_features])

# Climate features
temp_features = ["climate_Tm", "climate_Tx", "climate_Tn"]
sun_features = ["climate_S", "climate_SG"]
rh_features = ["climate_RH"]

restoration_dataset[temp_features] = scaler.fit_transform(restoration_dataset[temp_features])
restoration_dataset[sun_features] = scaler.fit_transform(restoration_dataset[sun_features])
restoration_dataset[rh_features] = scaler.fit_transform(restoration_dataset[rh_features])

# Precipitation: log-transform and scale
restoration_dataset["climate_P_log"] = np.log1p(restoration_dataset["climate_P"])
restoration_dataset["climate_P_scaled"] = scaler.fit_transform(
    restoration_dataset[["climate_P_log"]]
)


4. Distance and land cover scoring
    * Step 4 – Distance and Land Cover

<code>Distances to communities and roads are inverted: closer fires have higher restoration priority. Land cover classes are mapped to numeric scores based on ecological importance.</code>

In [62]:
# Distance features: closer = higher restoration priority
dist_features = ["dist_community", "dist_roads"]

# Scale distances
restoration_dataset[dist_features] = scaler.fit_transform(restoration_dataset[dist_features])

# Invert so smaller distance → higher score
for col in dist_features:
    restoration_dataset[col] = 1 - restoration_dataset[col]

# Land cover scoring: numeric importance based on ecological reasoning
# Example mapping: coniferous=1, mixed=0.7, deciduous=0.5, others=0.2
landcover_mapping = {
    "coniferous": 1.0,
    "mixed": 0.7,
    "deciduous": 0.5
}

restoration_dataset["land_cover_score"] = restoration_dataset["land_cover_class"].map(landcover_mapping).fillna(0.2)


5. Composite climate features (Temperature, Relative humidity and Temperature)
    * Step 5 – Climate Score

<code>Temperature, precipitation, solar radiation, and relative humidity are combined into a single weighted climate score for each fire area.</code>

In [63]:
# Weighted climate composite
restoration_dataset["climate_score"] = (
    0.25 * restoration_dataset["climate_Tm"] +
    0.25 * restoration_dataset["climate_Tx"] +
    0.25 * restoration_dataset["climate_Tn"] +
    0.15 * restoration_dataset["climate_P_scaled"] +
    0.05 * restoration_dataset["climate_S"] +
    0.05 * restoration_dataset["climate_SG"] +
    0.10 * restoration_dataset["climate_RH"]
)


6. Compute final restoration score
    * Step 6 – Creating the feature restoration_score to be the Machine Learning spine for pattern detection.

<code>All features are combined into a final weighted score. Clipping at the 1st and 99th percentiles avoids extreme outliers, producing a more interpretable distribution</code>

In [64]:
restoration_dataset["restoration_score"] = (
    0.26 * restoration_dataset[["burnsev_mean","burnsev_max","burnsev_min"]].mean(axis=1) +
    0.20 * restoration_dataset["fire_size_sqm"] +
    0.17 * restoration_dataset["dist_community"] +
    0.05 * restoration_dataset["dist_roads"] +
    0.12 * restoration_dataset["land_cover_score"] +
    0.10 * restoration_dataset["climate_score"] +
    0.05 * restoration_dataset[["elevation_mean","elevation_std"]].mean(axis=1) +
    0.05 * restoration_dataset["overlap_pct"]
)

# Clip final score to remove extreme outliers
restoration_dataset["restoration_score"] = restoration_dataset["restoration_score"].clip(
    lower=restoration_dataset["restoration_score"].quantile(0.01),
    upper=restoration_dataset["restoration_score"].quantile(0.99)
)

In [65]:
# Clip scores at 1st and 99th percentiles to avoid outliers
lower = restoration_dataset["restoration_score"].quantile(0.01)
upper = restoration_dataset["restoration_score"].quantile(0.99)

restoration_dataset["restoration_score"] = restoration_dataset["restoration_score"].clip(lower, upper)


In [66]:
restoration_dataset.columns

Index(['fire_id', 'fire_year', 'fire_month', 'fire_day', 'fire_rep_date',
       'fire_size_ha', 'fire_calc_ha', 'fire_cause', 'fire_map_method',
       'fire_poly_date', 'climate_index', 'dist_to_station', 'dist_roads',
       'overlap_pct', 'dist_community', 'elevation_mean', 'elevation_min',
       'elevation_max', 'elevation_std', 'burnsev_min', 'burnsev_max',
       'burnsev_mean', 'burnsev_std', 'climate_Tm', 'climate_Tx', 'climate_Tn',
       'climate_P', 'climate_S', 'climate_SG', 'climate_year', 'climate_month',
       'climate_long', 'climate_lat', 'climate_RH', 'land_cover_class',
       'geometry', 'fire_size_sqm', 'fire_calc_sqm', 'land_cover_score',
       'climate_score', 'restoration_score', 'fire_size_sqm_log',
       'climate_P_log', 'climate_P_scaled'],
      dtype='object')

In [80]:
restoration_dataset["restoration_score"] = restoration_dataset["restoration_score"].clip(lower=0)

In [81]:
restoration_dataset = gdf.to_file("../outputs/restoration_score_dataset.gpkg", driver="GPKG")

In [82]:
restoration_dataset = gpd.read_file("../outputs/restoration_score_dataset.gpkg")
restoration_dataset.head()

,fire_id,fire_year,fire_month,fire_day,fire_rep_date,fire_size_ha,fire_calc_ha,fire_cause,fire_map_method,fire_poly_date,...,climate_long,climate_lat,climate_RH,land_cover_class,fire_size_sqm,fire_calc_sqm,land_cover_score,climate_score,restoration_score,geometry
0,2004-C10176,2004,6,23,2004-06-23,520.7,520.796287,L,digitized,2007-05-17,...,-124.136,52.114,-0.665724,Unclassified,-0.036879,5.207963e+06,0.2,0.218898,0.166865,POINT (1103487.457 895235.59)
1,2004-C50114,2004,6,20,2004-06-20,268.2,268.290572,L,digitized,2007-05-17,...,-124.136,52.114,-0.665724,Developed,-0.078927,2.682906e+06,0.2,0.218898,0.161037,POINT (1059804.998 790281.638)
2,2004-C50125,2004,6,21,2004-06-21,20506.4,20506.415129,L,Modified from Protection,2007-05-17,...,-126.587,52.389,0.542111,Unclassified,3.291258,2.050642e+08,0.2,0.793427,0.895461,POINT (1027509.177 798970.739)
3,2004-C50149,2004,6,22,2004-06-22,2408.5,2408.587142,L,digitized,2007-05-17,...,-126.587,52.389,0.542111,Unclassified,0.277489,2.408587e+07,0.2,0.793427,0.400627,POINT (980057.152 883141.445)
4,2004-C50508,2004,8,16,2004-08-16,110.7,110.797330,L,digitized,2007-05-17,...,-126.587,52.389,0.542111,Rock/Rubble,-0.105154,1.107973e+06,0.2,0.793427,0.321202,POINT (981546.966 867012.646)


In [83]:
restoration_dataset.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 29955 entries, 0 to 29954
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fire_id            29955 non-null  object        
 1   fire_year          29955 non-null  int64         
 2   fire_month         29955 non-null  int64         
 3   fire_day           29955 non-null  int64         
 4   fire_rep_date      29955 non-null  datetime64[ms]
 5   fire_size_ha       29955 non-null  float64       
 6   fire_calc_ha       29955 non-null  float64       
 7   fire_cause         29955 non-null  object        
 8   fire_map_method    29935 non-null  object        
 9   fire_poly_date     28660 non-null  datetime64[ms]
 10  climate_index      29955 non-null  int64         
 11  dist_to_station    29955 non-null  float64       
 12  dist_roads         29955 non-null  float64       
 13  overlap_pct        29955 non-null  float64       
 14

In [85]:
restoration_dataset["restoration_score"].describe()

count    20349.000000
mean         0.300269
std          0.347355
min         -1.437790
25%          0.118820
50%          0.266201
75%          0.474275
max         17.669669
Name: restoration_score, dtype: float64

In [86]:
restoration_dataset["restoration_score"].quantile([0.01, 0.99])

0.01   -0.473604
0.99    0.985403
Name: restoration_score, dtype: float64

In [87]:
Q1 = restoration_dataset["restoration_score"].quantile(0.28)
Q3 = restoration_dataset["restoration_score"].quantile(0.72)
IQR = Q3 - Q1

# Define upper and lower bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


In [88]:
print(lower_bound)
print(upper_bound)

-0.302013273826378
0.8783675365934864


In [89]:
restoration_dataset = restoration_dataset[
    (restoration_dataset["restoration_score"] >= lower_bound) &
    (restoration_dataset["restoration_score"] <= upper_bound)
]


In [90]:
restoration_dataset["restoration_score"].describe()


count    19104.000000
mean         0.288011
std          0.255305
min         -0.301344
25%          0.124080
50%          0.262108
75%          0.442487
max          0.877925
Name: restoration_score, dtype: float64

In [91]:
restoration_dataset.shape

(19104, 41)